# Creating Vector Database

## Load Configurations & Libraries

In [1]:
## If we run this machine on GPU machine it will be faster.
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -Uq sentence-transformers

In [1]:
# @title Load Configuraiton 
import hbqaconfig
conf = hbqaconfig.setEnv('local')
# for k,v in conf.items(): print (k,":",v)

### Selecting and Loading Embedding Model.
- Different models are created on different kind of corpus
- Different vector size
- Some are normailzied, in that case dot product and cosine is same. Dot is less costly

In [62]:
!pip install huggingface
import huggingface

!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' cr

In [3]:
# https://www.sbert.net/docs/pretrained_models.html

#250MB, multi-qa-distilbert-cos-v1',  Max Sequence Length:	512, Dimensions:768, Normalized Embeddings:	true
#80MB, all-MiniLM-L6-v2, Max Sequence Length:	256, Dimensions:	384, Normalized Embeddings:	true
##290MB, all-distilroberta-v1, Max Sequence Length:	512, Dimensions:	768, Normalized Embeddings:	true
#420MB, all-mpnet-base-v2, Max Sequence Length:	384, Dimensions:	768, Normalized Embeddings:	true
##1.36GB, all-roberta-large-v1, Max Sequence Length:	256, Dimensions: 1024, Normalized Embeddings:	true

# embmodelname = 'multi-qa-distilbert-cos-v1'
# embmodelname = 'all-MiniLM-L6-v2'
embmodelname = 'all-distilroberta-v1'
embmodelname = 'multi-qa-mpnet-base-dot-v1'
embmodelname =  'all-roberta-large-v1'

embmodelname1 = "_" + embmodelname

In [3]:
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer, util

c:\Users\hari_\anaconda3\envs\hbqa\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
#util.dot_score, util.cos_sim

embmodel = SentenceTransformer(embmodelname)

# Util Functions

In [8]:
# When some function is decorated with this function. It will show the time to execute that function.
# we will use this calculate the time for embedding.
def get_time(func):
    import time
    def wrapper():
        start_time = time.time()

        result = func()

        end_time = time.time()
        time_spent=end_time-start_time

        print(f"Time Taken to Process {time_spent}")
        return result

    return wrapper

# Creating Chunk Embedding

## Load Chunk Dataset

In [9]:
FileToProcess = '04.2-Chunk_with_Prompts+ChatGPT_Response.csv' #This is based on Chapter 3 & 4.
df_hbqa_chunk= pd.read_csv(conf['PE_FOLDER']+FileToProcess) #This CSV is created from google drive file. It has Chunk info.

print(df_hbqa_chunk.shape)
df_hbqa_chunk = df_hbqa_chunk.loc[df_hbqa_chunk['ChatGPT_Response'].notna(),:]
df_hbqa_chunk = df_hbqa_chunk.loc[df_hbqa_chunk['ChatGPT_Response'].str.len()>10,:]
print(df_hbqa_chunk.shape)

(1697, 9)
(334, 9)


In [10]:
df_hbqa_chunk.tail(10)

,Section_Id,Chunk_Id,Chunk_Letters,Chunk_Words,Chunk_Approx_Tokens,Chunk,Prompt,ChatGPT_Response,1106
1527,Book05_039,909,1647,276,367,"Section XXXIX\n""Dhritarashtra said, 'Man is no...","Write 1 unique questions, corresponding answer...",Question 1: Why did Vidura advise King Dhritar...,NaN
1528,Book05_041,912,1989,334,444,"Section XLI\n(Sanat-sujata Parva)\n""Dhritarash...","Write 1 unique questions, corresponding answer...",Question 1: What doubt in Dhritarashtra's mind...,NaN
1529,Book05_057,935,2374,398,529,"""Sanjaya said, 'O Bharata, Dhrishtadyumna is a...","Write 1 unique questions, corresponding answer...",Question 1: Why did Dhrishtadyumna advise Sanj...,NaN
1530,Book05_067,945,1874,304,404,"Section LXVII\n""Vaisampayana said, 'When Duryo...","Write 1 unique questions, corresponding answer...",Question 1: Why did King Dhritarashtra seek Sa...,NaN
1531,Book05_068,946,2376,397,528,"Section LXVIII\n""Sanjaya said, 'Those adorable...","Write 1 unique questions, corresponding answer...",Question 1: How does Sanjaya describe the divi...,NaN
1532,Book05_071,949,1628,269,358,"Section LXXI\n""Dhritarashtra said, 'O Sanjaya,...","Write 1 unique questions, corresponding answer...","Question 1: In this passage, how does Dhritara...",NaN
1533,Book05_072,952,810,147,196,"""Yudhishthira said, 'Do, O Krishna, as it plea...","Write 1 unique questions, corresponding answer...",Question 1: What does Yudhishthira request of ...,NaN
1534,Book05_081,961,1383,229,305,"Section LXXXI\n""Sahadeva said, 'What hath been...","Write 1 unique questions, corresponding answer...",Question 1: What is Sahadeva's desire and the ...,NaN
1535,Book05_090,974,932,148,197,"Thus consoled by Krishna, Kunti, afflicted wit...","Write 1 unique questions, corresponding answer...",Question 1: How did Kunti respond to Krishna's...,NaN
1536,Book05_101,989,2160,327,435,"Section CI\n""Narada said, 'This region belonge...","Write 1 unique questions, corresponding answer...",Question 1: Can you provide the names of some ...,NaN


In [11]:
df_hbqa_chunk.reset_index(inplace=True)

In [12]:
df_hbqa_chunk.head(3)

,index,Section_Id,Chunk_Id,Chunk_Letters,Chunk_Words,Chunk_Approx_Tokens,Chunk,Prompt,ChatGPT_Response,1106
0,222,Book03_002,389,4793,808,1075,Even this is eternal morality. They that perfo...,"Write 5 unique questions, corresponding answer...",Question 1: What is the significance of perfor...,5.0
1,223,Book03_003,390,5453,851,1132,"Section III\n""Vaisampayana said, 'Yudhishthira...","Write 6 unique questions, corresponding answer...",Question 1: What predicament does Yudhishthira...,6.0
2,224,Book03_004,392,4383,775,1031,"Section IV\n""Vaisampayana said,--""After the Pa...","Write 5 unique questions, corresponding answer...",Question 1: What was the primary concern expre...,5.0


In [13]:
df_hbqa_chunk.columns

Index(['index', 'Section_Id', 'Chunk_Id', 'Chunk_Letters', 'Chunk_Words',
       'Chunk_Approx_Tokens', 'Chunk', 'Prompt', 'ChatGPT_Response', '1106'],
      dtype='object')

In [14]:
df_hbqa_chunk.drop(columns=['1106'], axis=1,inplace=True)

In [15]:
if torch.cuda.is_available():
    DEVICE = torch.device("cuda")
else:
    DEVICE = torch.device("cpu")

DEVICE

device(type='cuda')

In [16]:
# Convert Chunk colum values to List.
@get_time
def get_chunk_vector():
  Chunk_Sentences = df_hbqa_chunk.Chunk.tolist()
  Chunk_Embeddings = embmodel.encode(Chunk_Sentences)
  return Chunk_Embeddings

Chunk_Embeddings = get_chunk_vector()

Time Taken to Process 25.409409761428833


In [17]:
# What is the type of Chunk_Embeddings?
# type(Chunk_Embeddings)

In [18]:
# Convert These embedding into torch tensor and load into available DEVICE
ChunkVectors= torch.tensor(Chunk_Embeddings, dtype=torch.float).to(DEVICE)

In [19]:
# Flatten the tensors into 1D arrays
# ChunkVec_list = ChunkVectors.tolist()

In [20]:
# Flatten the tensors into 1D arrays/list
# Create ChunkVector in dataset.
df_hbqa_chunk['ChunkVector'+embmodelname1 ] = ChunkVectors.tolist()

In [21]:
print(df_hbqa_chunk.shape)
df_hbqa_chunk.head(3)

(334, 10)


,index,Section_Id,Chunk_Id,Chunk_Letters,Chunk_Words,Chunk_Approx_Tokens,Chunk,Prompt,ChatGPT_Response,ChunkVector_all-roberta-large-v1
0,222,Book03_002,389,4793,808,1075,Even this is eternal morality. They that perfo...,"Write 5 unique questions, corresponding answer...",Question 1: What is the significance of perfor...,"[0.039779193699359894, 0.043261121958494186, -..."
1,223,Book03_003,390,5453,851,1132,"Section III\n""Vaisampayana said, 'Yudhishthira...","Write 6 unique questions, corresponding answer...",Question 1: What predicament does Yudhishthira...,"[-0.02773909643292427, 0.03051523119211197, -0..."
2,224,Book03_004,392,4383,775,1031,"Section IV\n""Vaisampayana said,--""After the Pa...","Write 5 unique questions, corresponding answer...",Question 1: What was the primary concern expre...,"[0.05610743537545204, 0.04615481570363045, 0.0..."


In [22]:
# Save the Vector on harddisk.
df_hbqa_chunk.to_csv(conf['PE_FOLDER'] + '04.3-HBQA_Chunk_Vector'+embmodelname1+'.csv', index=None)
print(embmodelname)

all-roberta-large-v1


# Check whether a correct document can be retrived using a Question

In [33]:
df_hbqa_chunk = pd.read_csv(conf['PE_FOLDER'] + '04.3-HBQA_Chunk_Vector'+embmodelname1+'.csv')
(df_hbqa_chunk.shape)
df_hbqa_chunk.head(3)


,index,Section_Id,Chunk_Id,Chunk_Letters,Chunk_Words,Chunk_Approx_Tokens,Chunk,Prompt,ChatGPT_Response,ChunkVector_all-roberta-large-v1
0,222,Book03_002,389,4793,808,1075,Even this is eternal morality. They that perfo...,"Write 5 unique questions, corresponding answer...",Question 1: What is the significance of perfor...,"[0.039779193699359894, 0.043261121958494186, -..."
1,223,Book03_003,390,5453,851,1132,"Section III\n""Vaisampayana said, 'Yudhishthira...","Write 6 unique questions, corresponding answer...",Question 1: What predicament does Yudhishthira...,"[-0.02773909643292427, 0.03051523119211197, -0..."
2,224,Book03_004,392,4383,775,1031,"Section IV\n""Vaisampayana said,--""After the Pa...","Write 5 unique questions, corresponding answer...",Question 1: What was the primary concern expre...,"[0.05610743537545204, 0.04615481570363045, 0.0..."


In [34]:
import ast
len(ast.literal_eval(df_hbqa_chunk['ChunkVector'+embmodelname1][10]))

1024

In [35]:
chunk_id = df_hbqa_chunk['Chunk_Id'].to_list()

chunk_vector = [ast.literal_eval(df_hbqa_chunk['ChunkVector'+embmodelname1][i]) for i in range(df_hbqa_chunk.shape[0])]

In [36]:
len(chunk_vector), type(chunk_vector), len(chunk_vector[0])

(334, list, 1024)

In [37]:
chunk_vector = torch.tensor(chunk_vector)

In [38]:
len(chunk_vector), type(chunk_vector), len(chunk_vector[0])

(334, torch.Tensor, 1024)

In [42]:
from random import randint
i = randint(0, df_hbqa_chunk.shape[0])
df_hbqa_chunk.loc[i,:]

index                                                                             224
Section_Id                                                                 Book03_004
Chunk_Id                                                                          392
Chunk_Letters                                                                    4383
Chunk_Words                                                                       775
Chunk_Approx_Tokens                                                              1031
Chunk                               Section IV\n"Vaisampayana said,--"After the Pa...
Prompt                              Write 5 unique questions, corresponding answer...
ChatGPT_Response                    Question 1: What was the primary concern expre...
ChunkVector_all-roberta-large-v1    [0.05610743537545204, 0.04615481570363045, 0.0...
Name: 2, dtype: object

In [43]:
sep=":"
query = df_hbqa_chunk.loc[i,'ChatGPT_Response'].split('?')[0].split(sep)[1] +'?'
print(query)

# create the query vector
question_embedding = torch.tensor(embmodel.encode(query).tolist())
top_k = 10
hits = util.semantic_search(question_embedding, chunk_vector, top_k=top_k)

predictedId=[]
for hit in hits[0]:
  hbqa_chunk_idx = hit['corpus_id']
  print('Id:',hbqa_chunk_idx, 'Chunk_Id: ', df_hbqa_chunk.loc[hbqa_chunk_idx,'Chunk_Id'], ' Score:', hit['score'])
  predictedId.append(df_hbqa_chunk.loc[hbqa_chunk_idx,'Chunk_Id'])


 What was the primary concern expressed by Dhritarashtra to Vidura regarding the situation after the Pandavas had gone to the forest?
Id: 323 Chunk_Id:  896  Score: 0.6410800814628601
Id: 2 Chunk_Id:  392  Score: 0.6256968975067139
Id: 5 Chunk_Id:  395  Score: 0.608024001121521
Id: 4 Chunk_Id:  394  Score: 0.5505766868591309
Id: 272 Chunk_Id:  358  Score: 0.539655327796936
Id: 170 Chunk_Id:  690  Score: 0.5112076997756958
Id: 167 Chunk_Id:  687  Score: 0.5107508897781372
Id: 3 Chunk_Id:  393  Score: 0.5061450004577637
Id: 314 Chunk_Id:  783  Score: 0.4772453308105469
Id: 175 Chunk_Id:  697  Score: 0.475735068321228


In [44]:
predictedId

[896, 392, 395, 394, 358, 690, 687, 393, 783, 697]